# hw3 - ensembles

## 1 Подготовка данных

Загрузите и предобработайте данные (по своему усмотрению) из hw1

In [2]:
from google.colab import files
uploaded = files.upload()

Saving train_features_with_answers.csv to train_features_with_answers.csv


In [3]:
import pandas as pd
import numpy as np
import seaborn as sns

In [4]:
# Читаем данные

X_train = pd.read_csv('train_features_with_answers.csv')
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 454 entries, 0 to 453
Data columns (total 31 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   school      454 non-null    object 
 1   sex         454 non-null    object 
 2   age         426 non-null    float64
 3   address     449 non-null    object 
 4   famsize     454 non-null    object 
 5   Pstatus     454 non-null    object 
 6   Medu        454 non-null    int64  
 7   Fedu        454 non-null    int64  
 8   Mjob        454 non-null    object 
 9   Fjob        454 non-null    object 
 10  reason      454 non-null    object 
 11  guardian    454 non-null    object 
 12  traveltime  454 non-null    int64  
 13  studytime   454 non-null    int64  
 14  failures    454 non-null    int64  
 15  schoolsup   454 non-null    object 
 16  famsup      454 non-null    object 
 17  paid        454 non-null    object 
 18  activities  454 non-null    object 
 19  nursery     454 non-null    o

In [5]:
X_train['sex'] = X_train['sex'].where(X_train['sex'].isin(["F", "M"]), np.nan)

X_train['age'] = X_train['age'].where(
    ~X_train['age'].isna() & (X_train['age'] >= 15) & (X_train['age'] <= 22),
    np.nan
)

In [6]:
X_train = X_train.drop(columns=['Fedu', 'Dalc'])

In [7]:
def fill_nan_with_closest_match(df, column):
    def similarity(row1, row2, numeric_columns):
        diff = row1[numeric_columns] - row2[numeric_columns]
        return np.sqrt((diff**2).sum())

    numeric_columns = df.select_dtypes(include=[np.number]).columns.tolist()

    if column in numeric_columns:
        numeric_columns.remove(column)

    df_copy = df.copy()
    nan_indices = df[df[column].isna()].index

    for idx in nan_indices:
        current_row = df_copy.loc[idx]
        candidates = df_copy.drop(idx).dropna(subset=[column])

        similarities = candidates.apply(lambda row: similarity(current_row, row, numeric_columns), axis=1)

        closest_match_idx = similarities.idxmin()

        df_copy.at[idx, column] = df_copy.at[closest_match_idx, column]

    return df_copy

df = fill_nan_with_closest_match(X_train, 'sex')
df  = fill_nan_with_closest_match(df, 'age')
df = fill_nan_with_closest_match(df, 'address')

In [8]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse_output=False)

categorical = ["school", "sex", "address", "famsize", "Pstatus",
               "Mjob", "Fjob", "reason","guardian",
               "schoolsup", "famsup", "paid", "activities",
               "nursery", "higher", "internet", "romantic"]

encoded_array = encoder.fit_transform(df[categorical])
encoded_df = pd.DataFrame(encoded_array, columns=encoder.get_feature_names_out(categorical))
df = pd.concat([df.drop(categorical, axis=1), encoded_df], axis=1)

In [9]:
from sklearn.model_selection import StratifiedKFold
from imblearn.over_sampling import SMOTE, RandomOverSampler

X = df.drop(columns=['G3'])
y = df['G3']

smote = SMOTE(sampling_strategy={0: 20, 7: 20, 18: 20}, random_state=42, k_neighbors=5)
ros = RandomOverSampler(sampling_strategy={1: 10, 5: 10, 6: 10, 19: 10}, random_state=42) #их слишком мало для SMOTE


X_resampled, y_resampled = smote.fit_resample(X, y)
X_resampled, y_resampled = ros.fit_resample(X_resampled, y_resampled)

print("After Oversampling:", dict(zip(*np.unique(y_resampled, return_counts=True))))

After Oversampling: {0: 20, 1: 10, 5: 10, 6: 10, 7: 20, 8: 26, 9: 26, 10: 70, 11: 69, 12: 47, 13: 60, 14: 47, 15: 36, 16: 23, 17: 21, 18: 20, 19: 10}


In [10]:
#df.info()

## 2 Обоснуйте выбор слабых (базовых) алгоритмов

Попробуем использовать простые модели из hw1. Все они разные(что важно), но одни сильно коррелируют между собой, а некоторые не так сильно. Посмотрим, как ситуацию улучшат Стекинг и Блендинг. Интересно сравнить со взвешиванием, которое делали в хв1.
 Сначала посмотрим, как они работают по отдельности, а потом посмотрим как улучшит качество Blending и Stacking

In [11]:
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, cross_val_predict
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB


In [12]:
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

model_1 = LogisticRegression(C=0.1, max_iter=10000, random_state=42)
model_1.fit(X_train, y_train)
preds = model_1.predict(X_test).astype(np.int64)
print("LogisticRegression Accuracy test:", mean_squared_error(y_test, preds))
print("LogisticRegression Accuracy train", mean_squared_error(y_train, model_1.predict(X_train)))

RandomForestClassifier Accuracy test: 6.076190476190476
RandomForestClassifier Accuracy train 7.761904761904762


In [13]:
model_2 = SVC(C=0.5, kernel='linear', probability=True, random_state=42)
model_2.fit(X_train, y_train)
preds = model_2.predict(X_test)
print("SVC Accuracy test:", mean_squared_error(y_test, preds))
print("SVC Accuracy train", mean_squared_error(y_train, model_2.predict(X_train)))

GradientBoostingClassifier Accuracy test: 4.866666666666666
GradientBoostingClassifier Accuracy train 3.9404761904761907


In [14]:
model_3 = KNeighborsClassifier(n_neighbors=1)
model_3.fit(X_train, y_train)
preds = model_3.predict(X_test).astype(np.int64)
print("KNeighborsClassifier Accuracy test:", mean_squared_error(y_test, preds))
print("KNeighborsClassifier Accuracy train", mean_squared_error(y_train, model_3.predict(X_train)))

RandomForestClassifier Accuracy test: 7.914285714285715
RandomForestClassifier Accuracy train 0.0


In [16]:
model_4 = GaussianNB()
model_4.fit(X_train, y_train)
preds = model_4.predict(X_test).astype(np.int64)
print("GaussianNB Accuracy test:", mean_squared_error(y_test, preds))
print("GaussianNB Accuracy train", mean_squared_error(y_train, model_4.predict(X_train)))

RandomForestClassifier Accuracy test: 21.18095238095238
RandomForestClassifier Accuracy train 18.24047619047619


In [42]:
model_1 = LogisticRegression(C=0.1, max_iter=10000, random_state=42)
model_2 = SVC(C=0.5, kernel='linear', probability=True, random_state=42)
model_3 = KNeighborsClassifier(n_neighbors=1)
model_4 = GaussianNB()

models = [model_1, model_2, model_3, model_4]

X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.1, random_state=42)
X_train_train, X_val, y_train_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

## 3 Постройте решение на основе подхода Blending

Правила:
- Нужно использовать вероятности
- Предложите что-то лучше, чем брать среднее от предсказаний моделей (оценивать уверенность алгоритмов, точности и т.д.)
- Заставьте базовые алгоритмы быть некорелированными
- Добавьте рандома (например, стройте ваши алгоритмы на разных выборках, по разному предобрабатывайте данные или применяйте для разных признаков соответствующие алгоритмы ... )
- Проявите смекалку
- Цель: метрика MSE на тесте меньше 10

In [43]:
preds_val = []

for model in models:
  model.fit(X_train_train, y_train_train)
  preds_val.append(pd.Series(model.predict(X_val)))

# Создание метапризнаков для blending
meta_features = pd.concat(preds_val, axis=1)

# Метамодель
meta_model = LinearRegression()
meta_model.fit(meta_features, y_val)

preds_test = []

for model in models:
  preds_test.append(pd.Series(model.predict(X_test)))

#Создание новых тестовых данных
X_test = pd.concat(preds_test, axis=1)
# print(X_test)

# Оценка на тестовой выборке
blended_preds = meta_model.predict(X_test).astype(np.int64)
print("Blending MSE:", mean_squared_error(y_test, blended_preds))

Blending MSE: 4.113207547169812


In [44]:
model_1 = LogisticRegression(C=0.1, max_iter=10000, random_state=42)
model_2 = SVC(C=0.5, kernel='linear', probability=True, random_state=42)
model_3 = KNeighborsClassifier(n_neighbors=1)
model_4 = GaussianNB()

models = [model_1, model_2, model_3, model_4]

X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.1, random_state=42)
X_train_train, X_val, y_train_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

## 4 Постройте решение на основе подхода Stacking

Правила:
- Реализуйте пайплайн обучения и предсказания (например, sklearn.pipeline или класс)
- Проведите оптимизацию пайплайна
- Оцените вклад каждого базового алгоритма в итоговое предсказание
- Цель: метрика MSE на тесте меньше 10

In [45]:
preds_cv = []

for model in models:
  preds_cv.append(pd.Series(cross_val_predict(model, X_train, y_train, cv=5).ravel()))
  model.fit(X_train, y_train)

# Создание мета-признаков для blending
meta_features = pd.concat(preds_cv, axis=1)

# Обучение метамодели на мета-признаках
meta_model.fit(meta_features, y_train)

# Финальное предсказание на тестовых данных
test_preds = []
for model in models:
  test_preds.append(pd.Series(model.predict(X_test)))

X_test_final = pd.concat(test_preds, axis=1)
stacked_preds = meta_model.predict(X_test_final)

print("Stacking MSE:", mean_squared_error(y_test, stacked_preds))

Stacking MSE: 3.703677193151149


Интересный вывод: при использовании первых двух(LogisticRegression, SVM) или первых трех(LogisticRegression, SVM, knn) Блендинг и Стекинг дают качество хуже, чем лучшая из двух(трех) моделей.
Но при добавлении четвертой модели (Наивный Баес), который сам по себе дает ужасную метрику(худшую среди всех моделей), у нас заметно улучшается качество на Блендинге и Стекинге, и появляется смысл их использовать(качество лучше чем у лучшей из 4-х моделей). Почему так? Видимо потому, что первые три модели сильно коррелируют с коэффом 0.8-0.9(что мы выяснили в хв1), а вот Баес коррелирует с остальными моделями не так сильно.
Итак, мы проверили на практике, что имеет смысл использовать модели, которые не сильно коррелируют, даже если сами по себе они дают плохую метрику

## * Доп задание (не обязательно, но решение будет поощряться)

Правила:
- Постройте несколько сильных алгоритмов разного класса (это может быть бустинг, нейросеть, ансамбль слабых алгоритмов, алгоритм на статистике, что придумаете)
- Реализуйте "управляющий" алгоритм, который на основе входных данных будет выбирать, какой из  сильных алгоритмов запустить (не на основе их работы, а именно на основе данных)